# Crédito

<p style="text-align: center"><img src="./static/img/cover.png" width="500" height="500" alt="Imagem gráfica com nome crédito e análise."></p>


------
<b><p style="text-align: center"> Análise de Crédito</p></b>

Com o intuito de otimizar as decisões de crédito e reduzir o risco de inadimplência, realizaremos uma análise detalhada dos dados de crédito, utilizando SQL. A identificação de padrões e tendências entre as diferentes classes econômicas e sexos permitirá a criação de modelos de crédito mais precisos e eficazes.

Os dados utilizados nesta análise foram extraídos de um repositório [GitHub](https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset). A base de dados contém informações sobre as transações de crédito realizadas de periodo não indetificado, sendo 2.564 registros e 12 variáveis.

<br></br>

| Atributo | Descrição |
| --- | --- |
| idade | idade do cliente |
| sexo | sexo do cliente (F ou M) |
| dependentes | número de dependentes do cliente |
| escolaridade | nível de escolaridade do cliente |
| salario_anual | faixa salarial do cliente |
| tipo_cartao | tipo de cartão do cliente |
| qtd_produtos | quantidade de produtos comprados nos últimos 12 meses |
| iteracoes_12m | quantidade de iterações/transações nos últimos 12 meses |
| meses_inativo_12m | quantidade de meses que o cliente ficou inativo |
| limite_credito | limite de crédito do cliente |
| valor_transacoes_12m | valor das transações dos últimos 12 meses |
| qtd_transacoes_12m | quantidade de transações dos últimos 12 meses |


## Importação de Modulos

In [ ]:
!pip install pandas plotly

In [10]:
import os
import pandas as pd
import plotly.express as px

## Extração dos dados

In [14]:
DATA_DIR = "./data/Query/"
get_data = lambda file_name: pd.read_csv(os.path.join(DATA_DIR,file_name))

## EDA

### Visão geral

**Query:**
```sql
SELECT * from credito limit 10;
```

In [29]:
get_data('Query_5.csv')

,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,45,M,3,ensino medio,casado,$60K - $80K,blue,5,3,1,12691.51,1144.90,42
1,49,F,5,mestrado,solteiro,menos que $40K,blue,6,2,1,8256.96,1291.45,33
2,51,M,3,mestrado,casado,$80K - $120K,blue,4,0,1,3418.56,1887.72,20
3,40,F,4,ensino medio,na,menos que $40K,blue,3,1,4,3313.03,1171.56,20
4,40,M,3,sem educacao formal,casado,$60K - $80K,blue,5,0,1,4716.22,816.08,28
5,44,M,2,mestrado,casado,$40K - $60K,blue,3,2,1,4010.69,1088.07,24
6,51,M,4,na,casado,$120K +,gold,6,3,1,34516.72,1330.87,31
7,32,M,0,ensino medio,na,$60K - $80K,silver,2,2,2,29081.49,1538.32,36
8,37,M,3,sem educacao formal,solteiro,$60K - $80K,blue,5,0,2,22352.50,1350.14,24
9,48,M,2,mestrado,solteiro,$80K - $120K,blue,6,3,3,11656.41,1441.73,32


### Variedade de cartões oferecidos

**Query:**
```sql
SELECT DISTINCT tipo_cartao FROM credito
```

In [12]:
get_data('Query_9.csv')

,tipo_cartao
0,silver
1,blue
2,gold
3,platinum


### Valores extremos registrados: máximo e mínimo

**Query:**
```sql
select max(limite_credito) as "Maior Limite",min(limite_credito) as "Menor Limite" from credito
```

In [13]:
get_data('Query_1.csv')

,Maior Limite,Menor Limite
0,34516.99,1438.03


### Distribuição do valor do crédito por faixa de renda anual

**Query:**
```sql
SELECT avg(limite_credito) as "Média de credito", salario_anual as "Salário Anual" FROM credito where salario_anual != 'na' group by salario_anual
```

In [14]:
get_data('Query_4.csv')

,Média de credito,Salário Anual
0,17801.488,$120K +
1,14886.556,$80K - $120K
2,9096.028,$60K - $80K
3,5348.356,$40K - $60K
4,4099.476,menos que $40K


### Quanto mais ganha, mais gasta?

**Query:**
```sql
select avg(valor_transacoes_12m) as "Média de gasto em 12 meses",salario_anual from credito where salario_anual != 'na' group by salario_anual order by "Média de gasto em 12 meses" desc
```

In [15]:
get_data('Query_13.csv')

,Média de gasto em 12 meses,salario_anual
0,1862.7195,menos que $40K
1,1838.2643,$40K - $60K
2,1818.6364,$60K - $80K
3,1755.2499,$80K - $120K
4,1701.4652,$120K +



### Quanto mais ganha, mais compra?

**Query:**
```sql
select avg(qtd_transacoes_12m) as "Média de transações",salario_anual from credito where salario_anual != 'na' group by salario_anual order by "Média de transações" desc
```



In [16]:
get_data('Query_14.csv')

,Média de transações,salario_anual
0,42.825963,menos que $40K
1,42.064240,$40K - $60K
2,41.097561,$60K - $80K
3,40.446721,$80K - $120K
4,39.842342,$120K +


### Distribuição dos salários: comparando homens e mulheres

**Query:**
```sql
select count(sexo) as "Quantitativo",salario_anual,sexo from credito where salario_anual != 'na' group by sexo,salario_anual order by salario_anual
```

In [17]:
df = get_data('Query_12.csv')

In [18]:
df.rename(columns={"salario_anual":"Salário Anual","sexo":"Sexo"},inplace=True)
df['Salário Anual'] = df['Salário Anual'].apply(lambda salary: salary.replace('-','a').replace('+','ou mais').replace('$','').replace('K',' mil').title())
df.sort_values('Quantitativo',ascending=False,inplace=True)

In [19]:
df.head()

,Quantitativo,Salário Anual,Sexo
5,597,Menos Que 40 Mil,F
4,488,80 Mil A 120 Mil,M
3,451,60 Mil A 80 Mil,M
1,274,40 Mil A 60 Mil,M
0,222,120 Mil Ou Mais,M


In [20]:
px.bar(df,x='Salário Anual',y='Quantitativo',color='Sexo',color_discrete_sequence=["#ff6361","#58508d"],title="Distribuição dos salários: comparando homens e mulheres")

### Proporção de clientes por gênero

**Query:**
```sql
SELECT count(*),sexo from credito group by sexo order by count(*) desc
```

In [21]:
df = get_data('Query_10.csv')

In [22]:
df.head()

,Quantitativo,sexo
0,1563,M
1,1001,F


In [23]:
px.pie(df,values='Quantitativo',names='sexo',color_discrete_sequence=["#58508d","#ff6361"],title="Distribuição de clientes por sexo",hole=0.4,labels={"sexo":"Sexo"})

### Distribuição etária por sexo

**Query:**
```sql
select avg(idade) as "Média de Idade",max(idade) as "Idade Máxima",min(idade) as "Idade Mínima",sexo from credito group by sexo
```

In [24]:
get_data('Query_6.csv')

,Média de Idade,Idade Máxima,Idade Mínima,sexo
0,46.429570,67,26,F
1,45.573896,73,26,M


### Distribuição das compras por sexo

**Query:**
```sql
select avg(qtd_transacoes_12m) as "Média de Transações",sexo from credito group by sexo order by avg(qtd_transacoes_12m) desc
```

In [25]:
get_data('Query_3.csv')

,Média de Transações,sexo
0,42.283716,F
1,41.333333,M


### Relação entre gênero e condições de crédito

**Query:**
```sql
select avg(limite_credito) as "Média de Crédito",sexo from credito group by sexo order by avg(limite_credito) desc
```

In [26]:
df = get_data('Query_2.csv')

In [27]:
df.head()

,Média de Crédito,sexo
0,11201.3570,M
1,5748.9844,F


In [28]:
px.bar(df,x='sexo',y='Média de Crédito',color='Média de Crédito',title='Média de crédito fornecido por sexo')

## Conclusão 

<aside>
💡 Descobertas a partir da visualização dos dados.
</aside>

- Observamos um desequilíbrio de gênero na amostra, com uma predominância significativa do sexo masculino (61%) em relação ao feminino (39%). Essa discrepância pode influenciar os resultados da análise e merece atenção especial.

- Há uma clara correlação positiva entre o valor do crédito concedido e a renda do solicitante. Ou seja, indivíduos com maiores rendimentos tendem a obter créditos de maior valor.

- A análise sugere que o modelo de concessão de crédito atual pode estar privilegiando indivíduos com maiores rendas. Essa abordagem pode estar excluindo segmentos da população com menores rendimentos, que, apesar de apresentarem um volume maior de transações e gastos com cartão de crédito, têm acesso a menores valores de crédito.

- A análise sugere que o modelo de concessão de crédito atual pode estar privilegiando indivíduos com maiores rendas. Essa abordagem pode estar excluindo segmentos da população com menores rendimentos, que, apesar de apresentarem um volume maior de transações e gastos com cartão de crédito, têm acesso a menores valores de crédito.
